In [2]:
import matplotlib.pyplot as plt
import cv2
import dlib
import numpy as np

In [8]:
# set up if using dnn
# prototxtPath = r"face_detector/deploy.prototxt"
# weightsPath = r"face_detector/res10_300x300_ssd_iter_140000_fp16.caffemodel"
# net = cv2.dnn.readNetFromCaffe(prototxtPath, weightsPath)
PREDICTOR_PATH = r"face_detector/shape_predictor_68_face_landmarks.dat"
LIP_DIST_CUTOFF = 5.0

detector = dlib.get_frontal_face_detector()

cap = cv2.VideoCapture(0)
# fourcc = cv2.VideoWriter_fourcc(*'XVID') 
# out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480)) 

predictor = dlib.shape_predictor(PREDICTOR_PATH)
while True:
    success, image = cap.read()
    h, w = image.shape[:2]
    
    # detector usage if using dnn
    # blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    # net.setInput(blob)
    # detections = net.forward()
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    # draw bounding box
    for face in faces:
        x, y, w, h = (face.left(), face.top(), face.width(), face.height())
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        dlib_rect = dlib.rectangle(int(x), int(y), int(x + w), int(y + h))
        landmarks = np.matrix([[p.x, p.y]  
               for p in predictor(image, dlib_rect).parts()])
        mouth_point_up = landmarks[62]
        mouth_point_down = landmarks[66]
        # print out landmark for upper lip and lower lip
        counter = 0
        for point in (mouth_point_up, mouth_point_down):  
            pos = (point[0, 0], point[0, 1])
            cv2.circle(image, pos, 2, color=(0, 255, 255), thickness=-1)
            counter+= 1
        dist = np.linalg.norm(mouth_point_up-mouth_point_down)
        if dist > LIP_DIST_CUTOFF:
            cv2.putText(image, "speaking!", (x, y - 10),  
               fontFace=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,  
               fontScale=1,  
               color=(0, 0, 255))

    # bounding box detection if using dnn
    # for i in range(detections.shape[2]):
    #     confidence = detections[0, 0, i, 2]
    #     if confidence > 0.5:
    #         x1 = int(detections[0, 0, i, 3] * w)
    #         y1 = int(detections[0, 0, i, 4] * h)
    #         x2 = int(detections[0, 0, i, 5] * w)
    #         y2 = int(detections[0, 0, i, 6] * h)
    #         cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)

    cv2.imshow("Webcam", image) # This will open an independent window
    out.write(image)
    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        cap.release()
        break

# out.release()  
cv2.destroyAllWindows() 
cv2.waitKey(1)

-1